# Multi-layer convolutional neural network model for facial expression recognition

Note: This program is modified from this tutorial: https://www.tensorflow.org/versions/r0.10/tutorials/mnist/pros/index.html, and it is used as a proof-of-concept for our project.

Required packages are:
* Tensorflow
* NumPy
* Scipy
* Opencv

In [29]:
from utils import *
import tensorflow as tf
import numpy as np
import cPickle as pickle

In [2]:
# Path of summary files for Tensorboard
# Note:
# After viewing the summaries in Tensorboard, delete this directory. Because
# the next time you run the code, all summaries will be appended to the same location,
# which will mess up the data.
log_path = '/tmp/facial_log'

# Load dataset

In [3]:
dataset, label_index, label_tag = load_data('dataset')
label_onehot = one_hot(label_index, 10)
print dataset.shape
print label_index
print label_tag
print label_onehot

(24, 300, 200, 3)
[ 5.  1.  3.  4.  4.  2.  1.  1.  4.  4.  4.  1.  4.  2.  4.  4.  4.  4.
  1.  4.  4.  1.  4.  4.]
['anger', 'disgust', 'fear', 'happiness', 'sadness']
[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.

# Splite data into trainning set and test set

In [31]:
train_x = dataset[0:18, :, :, :]
train_y = label_onehot[0:18]
test_x = dataset[18:, :, :, :]
test_y = label_onehot[18:]

# Define some shortcuts

In [5]:
def weight_variable(shape):
    """ Weight initialization """
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """ Bias initialization """
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    """ Convolution operation """
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """ Pooling """
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

# Build the model

In [7]:
# Input Layer
with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32, shape=[None, image_height, image_width, image_channel])
    y_ = tf.placeholder(tf.float32, shape=[None, 10])

# First Convolutional Layer
with tf.name_scope('ConvL1'):
    W_conv1 = weight_variable([5, 5, image_channel, 32])
    b_conv1 = bias_variable([32])
    x_image = tf.reshape(x, [-1,image_height,image_width,image_channel])
#    x_image = x
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool = max_pool_2x2(h_conv1)

# Second Convolutional Layer
with tf.name_scope('ConvL2'):
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

# Densely Connected Layer
with tf.name_scope('DenselyConnectedLayer'):
#    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    W_fc1 = weight_variable([75 * 50 * 64, 1024])
    b_fc1 = bias_variable([1024])
#    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 75*50*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
with tf.name_scope('Dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Readout Layer
with tf.name_scope('ReadoutLayer'):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# Lost Function
with tf.name_scope('LostFunction'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))

In [8]:
# Create summary entries
# Important note: do not add entries after tf.merge_all_summaries, otherwise the program will crashed!
tf.scalar_summary('loss-cross-entropy', cross_entropy)

<tf.Tensor 'ScalarSummary:0' shape=() dtype=string>

# Train the model

In [9]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# This is for gathering statistics to be write to the summary
summary_op = tf.merge_all_summaries()

# Run the session

In [34]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    writer = tf.train.SummaryWriter(log_path, graph=sess.graph)
    for i in range(50):
#        batch = mnist.train.next_batch(50)
        batch = next_batch(train_x, train_y, 5)
#        if i%100 == 0:
#            train_accuracy = sess.run(accuracy, feed_dict={
#                x:batch[0], y_: batch[1], keep_prob: 1.0})
#            print("step %d, training accuracy %g"%(i, train_accuracy))
        _, summary = sess.run([train_step, summary_op], feed_dict=
                        {x: batch[0], y_: batch[1], keep_prob: 0.5})
        writer.add_summary(summary, i)
        
    test_accuracy = sess.run(accuracy, feed_dict={
            x: test_x, y_: test_y, keep_prob: 1.0
        })
    print 'Accuracy on test set: ', test_accuracy

    # Gather the parameters
    w1 = sess.run(W_conv1)
    b1 = sess.run(b_conv1)
    w2 = sess.run(W_conv2)
    b2 = sess.run(b_conv2)
    wfc1 = sess.run(W_fc1)
    bfc1 = sess.run(b_fc1)
    wfc2 = sess.run(W_fc2)
    bfc2 = sess.run(b_fc2)

Accuracy on test set:  0.333333


# Write parameters to disk for reuse

In [36]:
# The data takes about 1GB..
#with open('parm.dat', 'wb') as f:
#    pickle.dump([w1, b1, w2, b2, wfc1, bfc1, wfc2, bfc2], f, pickle.HIGHEST_PROTOCOL)